# XGBoost regression (multi-node with Dask)

<table>
    <tr>
        <td>
            <img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" width="300">
        </td>
        <td>
            <img src="https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png" width="300">
        </td>
    </tr>
</table>

In [ ]:
import os
import warnings

import cloudpickle
import dask.dataframe as dd
import dask_xgboost

from dask.distributed import Client, wait
from dask_ml.metrics import mean_squared_error
from dask_saturn import SaturnCluster

warnings.simplefilter("ignore")

In [ ]:
MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
numeric_feat = [
    'pickup_weekday',
    'pickup_weekofyear',
    'pickup_hour',
    'pickup_week_hour',
    'pickup_minute',
    'passenger_count',
]
categorical_feat = [
    'PULocationID',
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'tip_fraction'

# Initialize Dask cluster

In [ ]:
n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers,
    scheduler_size='medium',
    worker_size='large',
    nthreads=2
)
client = Client(cluster)
cluster

Open the dashboard (link above ^) and watch it when you execute some commands, you'll see which tasks are running across the cluster.

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster from the "Dask" page in Saturn if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

# Load data and feature engineering

Load a sample from a single month for this exercise

In [ ]:
taxi = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
    assume_missing=True,
).sample(frac=0.3, replace=False)

In [ ]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB')

In [ ]:
def prep_df(df: dd.DataFrame) -> dd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi_train = prep_df(taxi)

In [ ]:
taxi_train.head()

Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing once.

In [ ]:
%%time
taxi_train = taxi_train.persist()
_ = wait(taxi_train)

# Train a model

In [ ]:
xgb_reg = dask_xgboost.XGBRegressor(
    objective="reg:squarederror",
    tree_method='approx',
    learning_rate=0.1,
    max_depth=8,
    n_estimators=100,
)

In [ ]:
%%time
_ = xgb_reg.fit(taxi_train[features], y=taxi_train[y_col])

## Save model

In [ ]:
with open(f'{MODEL_PATH}/xgboost_dask.pkl', 'wb') as f:
    cloudpickle.dump(xgb_reg, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
    assume_missing=True,
).sample(frac=0.01, replace=False)

taxi_test = prep_df(taxi_test)

The metrics functions in `dask_ml` can compute metrics on Dask collection like Dask Array and Dask DataFrame, so you never have to hold all of the test data in memory on the client.

In [ ]:
preds = xgb_reg.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)